In [90]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import ssl
import re
import time


In [2]:
def loadFiles():
    with open('restMasterList.json') as fp:
        f = json.load(fp)
        id = 20047
        print(len(f[str(id)]) > 0)
    return

In [3]:
def prepareMasterSeedList(district_elements, cuisine_elements, foodtype_elements):
    MasterSeedURL = []
    baseURL = "http://etw.nextdigital.com.hk/search/restaurant/?"

    
    for i in range(len(district_elements)):
        d = baseURL + "district=" + district_elements[i]
        MasterSeedURL.append(d)

    for i in range(len(cuisine_elements)):
        d = baseURL + "cuisine=" + cuisine_elements[i]
        MasterSeedURL.append(d)

    for i in range(len(foodtype_elements)):
        d = baseURL + "foodtype=" + foodtype_elements[i]
        MasterSeedURL.append(d)   

    print("Masterlist length: " +  str(len(MasterSeedURL)))
        
        
    return MasterSeedURL

In [4]:
def fetchSearchInfo(url):
    
    browser = webdriver.Chrome('/Users/samuelpun_old/Downloads/chromedriver')

    browser.get(url)
    time.sleep(1)

    elem = browser.find_element_by_tag_name("body")

        
    # extract Search Options seeds
    district_elements = extractElements(browser, "selected_district", "value", "find_elements_by_name")
    cuisine_elements = extractElements(browser, "selected_cuisine", "value", "find_elements_by_name")
    foodtype_elements = extractElements(browser, "selected_foodtype", "value", "find_elements_by_name")
    
    browser.close()
                                             
    return district_elements, cuisine_elements, foodtype_elements

In [5]:
def extractElements(browserObj, tagName, attr, method):
    
    if method == "find_elements_by_name":
        elements = browserObj.find_elements_by_name(tagName)
    elif method  == "find_elements_by_class_name":
        elements = browserObj.find_elements_by_class_name(tagName)
        
    snippets = []
    
    for item in elements:
        snippets.append(item.get_attribute(attr))
    
    return snippets

In [6]:
def crawlRestInfo(url, tag):
    
    browser = webdriver.Chrome('/Users/samuelpun_old/Downloads/chromedriver')

    browser.get(url)
    time.sleep(1.5)

    # extract restaurants class
    restclass_elements = extractElements(browser, "content-wrapper", 
                                             "innerHTML", "find_elements_by_class_name")
    browser.close()
        
                                             
    return restclass_elements

In [7]:
def processRestsData(restsEntry):
    
    count = 0
    
    # from a list of href exact restaurant and new masterList info
    for i in range(len(restsEntry)):
        s = str(restsEntry[i])
        restInfo = re.findall('href\\=\\"\\/zh\\/hongkong\\/(r\\-\S+\\-r[0-9]+)\\"',s)
        restAddr = re.findall('\\/district/\S+\\"\\>(\S+)\\<\\/a\\>(\S+)',s)
        posRate = re.findall('score score\\-big highlight\\"\\>([0-9]+)\\<\\/span\\>',s)
        negRate = re.findall('score highlight\\"\\>([0-9]+)\\<\\/span\\>',s)
        
        rest = {}
        
        
        rest = {}
        for item in restInfo:
            info = (urllib.parse.unquote(item)).split('-')
            if len( info[5]) > 0:
                rest['ID'] = info[5]
            if len( info[1]) > 0:
                rest['name'] = info[1]
            if len( info[2]) > 0:  
                rest['district'] = info[2]
            if len( info[3]) > 0: 
                rest['cuisine'] = info[3]
            if len( info[4]) > 0: 
                rest['foottype'] = info[4]
            if len( restAddr[0][1]) > 0: 
                rest['address'] = restAddr[0][1]
            if len(negRate) > 0:   
                rest['posRate'] = posRate
            if len(posRate) > 0: 
                rest['negRate'] = negRate
            
            if rest['ID'] in restMasterList:
                continue
            else:
                restMasterList[rest['ID']] = rest
                count = count + 1
    
    print(str(count) + " records updated. Total record: " + str(len(restMasterList)))

    # save file as JSON format
    with open('./ORrestMasterList.json', 'w+') as fp:
        json.dump(restMasterList, fp)
        #existingRecord = dict(json.load(fp))
         
    return restMasterList

In [8]:
def processCrawlList(restMasterList, seedMasterList):
    
    pageDownNumber = 1000
    
    # scan thru the seedMasterList go get a list of href
    restsEntry = []
    count = 0
    
    for i in range(len(seedMasterList)):
        print("fetching " + str(i) + "th URL out of " + str(len(seedMasterList)))
        this_url = seedMasterList[i]
        restclass_elements = crawlRestInfo(this_url, 'a', pageDownNumber, True) 
        restMasterList = processRestsData(restclass_elements)
        #restsEntry =  restsEntry + restclass_elements
        count = count + 1
    
    print(str(count) + " url fetched")
    
    #restMasterList = processRestsData(restsEntry)
        
    return restsEntry, restMasterList

In [10]:
#url = input('Enter - ')
seedMasterList = []
restMasterList = {}
seedURL = "http://etw.nextdigital.com.hk"
url = 'https://www.openrice.com/zh/hongkong/restaurants?what=%E6%97%A5%E6%9C%AC%E8%8F%9C'
seedMasterList.append(url)

#district_elements, cuisine_elements, foodtype_elements = fetchSearchInfo(seedURL)
#seedMasterList = prepareMasterSeedList(district_elements, cuisine_elements, foodtype_elements)

#restsEntry, restMasterList = processCrawlList(restMasterList, seedMasterList)

In [92]:
browser = webdriver.Chrome('/Users/samuelpun_old/Downloads/chromedriver')

browser.get(url)
time.sleep(1.5)



In [93]:
elem = browser.find_elements_by_class_name("pagination")
snippet = elem[0].get_attribute('innerHTML')
print(snippet)



    

    
    
    <a class="pagination-button     
    pagination-button-focus curr js-curr
    
     
     
" data-curr-url="/zh/hongkong/restaurants?what=%E6%97%A5%E6%9C%AC%E8%8F%9C" data-page="1">
        1
    </a>
    
    <a class="pagination-button     
     
     next js-next 
" href="/zh/hongkong/restaurants?what=%E6%97%A5%E6%9C%AC%E8%8F%9C&amp;page=2" data-page="2">
        2
    </a>
    
    <a class="pagination-button     
     
     next js-next 
" href="/zh/hongkong/restaurants?what=%E6%97%A5%E6%9C%AC%E8%8F%9C&amp;page=3" data-page="3">
        3
    </a>
    
    <a class="pagination-button     
     
     next js-next 
" href="/zh/hongkong/restaurants?what=%E6%97%A5%E6%9C%AC%E8%8F%9C&amp;page=4" data-page="4">
        4
    </a>
    
    <a class="pagination-button     
     
     next js-next 
" href="/zh/hongkong/restaurants?what=%E6%97%A5%E6%9C%AC%E8%8F%9C&amp;page=5" data-page="5">
        5
    </a>
    
    

    
    <a class="pagination-button next js-next" 